# 对象
本页上的示例演示了如何在 Weaviate 中创建单个对象。


In [7]:
## 测试
import json
import weaviate
from weaviate.auth import AuthApiKey

# 连接到本地部署的 Weaviate
client = weaviate.Client(
    url="http://127.0.0.1:8080",
    auth_client_secret=AuthApiKey("WVF5YThaHlkYwhGUSmCRgsX3tD5ngdN8pkih")
)

In [2]:
##V3

uuid = client.data_object.create(
    class_name="JeopardyQuestion",
    data_object={
        "question": "This vector DB is OSS & supports automatic property type inference on import",
        # "answer": "Weaviate",  # schema properties can be omitted
        "newProperty": 123,  # will be automatically added as a number property
    }
)

print(uuid)  # the return value is the object's UUID

fccecc40-85ef-44af-ae99-89274babd763


In [ ]:
### V4
jeopardy = client.collections.get("JeopardyQuestion")

uuid = jeopardy.data.insert({
    "question": "This vector DB is OSS & supports automatic property type inference on import",
    # "answer": "Weaviate",  # properties can be omitted
    "newProperty": 123,  # will be automatically added as a number property
})

print(uuid)  # the return value is the object's UUID

## 创建具有指定向量
创建对象时，您可以提供一个向量。（有关指定多个命名向量，请参阅下文。）

In [3]:
uuid = client.data_object.create(
    class_name="JeopardyQuestion",
    data_object={
        "question": "This vector DB is OSS and supports automatic property type inference on import",
        "answer": "Weaviate",
    },
    vector=[0.12345] * 1536
)

print(uuid)  # the return value is the object's UUID

3e8346aa-0c95-433b-8fb3-32aac342f8c0


In [ ]:
### V4
jeopardy = client.collections.get("JeopardyQuestion")
uuid = jeopardy.data.insert(
    properties={
        "question": "This vector DB is OSS and supports automatic property type inference on import",
        "answer": "Weaviate",
    },
    vector=[0.12345] * 1536
)

print(uuid)  # the return value is the object's UUID

## 创建具有命名向量
当你创建一个对象时，你可以指定命名向量（如果在你的集合中配置了）。

In [ ]:
### v3不支持
### V4

reviews = client.collections.get("WineReviewNV")  # This collection must have named vectors configured
uuid = reviews.data.insert(
    properties={
        "title": "A delicious Riesling",
        "review_body": "This wine is a delicious Riesling which pairs well with seafood.",
        "country": "Germany",
    },
    # Specify the named vectors, following the collection definition
    vector={
        "title": [0.12345] * 1536,
        "review_body": [0.31313] * 1536,
        "title_country": [0.05050] * 1536,
    }
)

print(uuid)  # the return value is the object's UUID

## 创建具有指定ID


In [4]:
##V3
uuid = client.data_object.create(
    class_name="JeopardyQuestion",
    data_object={
        "question": "This vector DB is OSS and supports automatic property type inference on import",
        "answer": "Weaviate",
    },
    uuid="12345678-e64f-5d94-90db-c8cfa3fc1234"
)

In [5]:
uuid

'12345678-e64f-5d94-90db-c8cfa3fc1234'

In [ ]:
### V4
properties = {
    "question": "This vector DB is OSS and supports automatic property type inference on import",
    "answer": "Weaviate",
}
jeopardy = client.collections.get("JeopardyQuestion")
uuid = jeopardy.data.insert(
    properties=properties,
    uuid="12345678-e64f-5d94-90db-c8cfa3fc1234"
)

print(uuid)  # the return value is the object's UUID

## 生成确定性ID
您可以根据您的数据对象生成一个 ID。
> 对象 ID 不是随机生成的。相同的值始终会生成相同的 ID。
> 如果您提供重复的 ID，Weaviate 会抛出错误。使用确定性 ID 可避免插入重复的对象。



In [6]:
from weaviate.util import generate_uuid5  # Generate a deterministic ID

data_object = {
    "question": "This vector DB is OSS and supports automatic property type inference on import",
    "answer": "Weaviate",
}
uuid = client.data_object.create(
    class_name="JeopardyQuestion",
    data_object=data_object,
    uuid=generate_uuid5(data_object),
)
print(uuid)

c9f915f8-8449-5c86-ad95-5497f1b7886c


In [ ]:
### V4
from weaviate.util import generate_uuid5  # Generate a deterministic ID

data_object = {
    "question": "This vector DB is OSS and supports automatic property type inference on import",
    "answer": "Weaviate",
}

jeopardy = client.collections.get("JeopardyQuestion")
uuid = jeopardy.data.insert(
    properties=data_object,
    uuid=generate_uuid5(data_object),
)

## 创建具有交叉引用的对象
您可以创建一个与其他对象交叉引用的对象。

In [ ]:
# data_obj is a dictionary with the object's properties
data_obj["hasCategory"] = [  # Add one or more cross-references through the "hasCategory" property
    {"beacon": f"weaviate://localhost/JeopardyCategory/{category_uuid}"}
]

data_uuid = client.data_object.create(
    data_obj,
    "JeopardyQuestion",
)


###V4
questions = client.collections.get("JeopardyQuestion")

questions.data.insert(
    properties=properties,  # A dictionary with the properties of the object
    uuid=obj_uuid,  # The UUID of the object
    references={"hasCategory": category_uuid},  # e.g. {"hasCategory": "583876f3-e293-5b5b-9839-03f455f14575"}
)

## 创建前验证对象
在创建对象之前，您可以根据集合定义来验证它。

In [8]:
result = client.data_object.validate(
    class_name="JeopardyQuestion",
    data_object={
      "question": "This vector DB is open-source and supports auto-schema",
      "answer": "Weaviate",
      "thisPropShouldNotEndUpInTheSchema": -1,
    },
    uuid="12345678-1234-1234-1234-123456789012",
)

# "invalid object: no such prop with name "thisPropShouldNotEndUpInTheSchema" found..."
print(json.dumps(result, indent=2))

## V4 不支持

{
  "error": [
    {
      "message": "invalid object: no such prop with name 'thisPropShouldNotEndUpInTheSchema' found in class 'JeopardyQuestion' in the schema. Check your schema files for which properties in this class are available"
    }
  ],
  "valid": false
}


## 多个向量嵌入（命名向量）
> 在 v1.24.0 中添加             
> Weaviate 集合支持多个命名向量。